Instalando pacotes

Importação

In [ ]:
!pip install requests beautifulsoup4 pandas numpy

### Web scraping EMBRAPA
Utilizando parâmetros de URL e na tabela 'tb_base tb_dados'


In [ ]:
# Bibliotecas necessárias para requisição, parsing HTML, manipulação de dados e controle de arquivos
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import os
from pathlib import Path

# ---------------------- CONFIGURAÇÕES INICIAIS ----------------------

# Define o diretório onde os arquivos CSV serão salvos
DATA_DIR = "data/raw"
# Cria o diretório se ele ainda não existir
Path(DATA_DIR).mkdir(parents=True, exist_ok=True)

# URL base do site da Embrapa Vitibrasil
base_url = "http://vitibrasil.cnpuv.embrapa.br/index.php"
# Intervalo de anos para coleta dos dados
anos = range(2008, 2024)

# Dicionário com as páginas do site a serem raspadas
# Cada entrada contém: nome do arquivo, se o ano é necessário e subopções da página (se houver)
config_paginas = {
    "opt_02": {
        "nome_arquivo": "producao",
        "requer_ano": True,
        "subopcoes": [None]
    },
    "opt_03": {
        "nome_arquivo": "processamento",
        "requer_ano": True,
        "subopcoes": [f"subopt_{i:02d}" for i in range(1, 3)]
    },
    "opt_04": {
        "nome_arquivo": "comercializacao",
        "requer_ano": True,
        "subopcoes": [None]
    },
    "opt_05": {
        "nome_arquivo": "importacao",
        "requer_ano": True,
        "subopcoes": [f"subopt_{i:02d}" for i in range(1, 3)]
    },
    "opt_06": {
        "nome_arquivo": "exportacao",
        "requer_ano": True,
        "subopcoes": [f"subopt_{i:02d}" for i in range(1, 3)]
    }
}

# ---------------------- FUNÇÕES PRINCIPAIS ----------------------

def scrape_tabelas(base_url, params):
    """
    Realiza requisição HTTP ao site com os parâmetros fornecidos
    e retorna as tabelas HTML encontradas com a classe 'tb_base tb_dados'
    """
    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        tabelas = soup.find_all('table', {'class': 'tb_base tb_dados'})
        return tabelas, soup
    except Exception as e:
        print(f"Erro ao acessar {params}: {str(e)}")
        return [], None

def processar_tabela_com_itens(soup, tabela):
    """
    Processa tabelas que possuem estrutura hierárquica com 'item' e 'subitem'
    Retorna um DataFrame estruturado mantendo a relação entre eles
    """
    linhas = tabela.find_all('tr')
    dados = []
    current_item = None

    for linha in linhas:
        item = linha.find('td', {'class': 'tb_item'})
        if item:
            current_item = item.get_text(strip=True)
            continue

        subitem = linha.find('td', {'class': 'tb_subitem'})
        if subitem and current_item:
            celulas = [td.get_text(strip=True) for td in linha.find_all('td')]
            dados.append([current_item] + celulas)

    if dados:
        num_colunas = max(len(linha) for linha in dados)
        colunas = ['Item'] + [f'Coluna_{i}' for i in range(1, num_colunas)]
        df = pd.DataFrame(dados, columns=colunas)
    else:
        # Caso a tabela não siga o padrão esperado, usa leitura direta
        df = pd.read_html(str(tabela))[0]

    return df

def tratar_valores_nulos(df):
    """
    Substitui valores inválidos ('-', 'nd') por 0 e tenta converter colunas para numérico
    """
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].replace('-', '0')
            df[col] = df[col].replace('nd', '0')
            try:
                df[col] = pd.to_numeric(df[col], errors='ignore')
            except:
                pass
    return df

def arquivo_existe(nome_arquivo):
    """
    Verifica se o arquivo CSV já existe no diretório de saída
    """
    caminho_completo = os.path.join(DATA_DIR, nome_arquivo)
    return os.path.exists(caminho_completo)

# ---------------------- FLUXO PRINCIPAL ----------------------

# Itera sobre as páginas configuradas
for opcao, config in config_paginas.items():
    # Gera o nome do arquivo de saída baseado no nome e no intervalo de anos
    if config["requer_ano"]:
        nome_arquivo = f"{config['nome_arquivo']}_{min(anos)}_{max(anos)}.csv"
    else:
        nome_arquivo = f"{config['nome_arquivo']}.csv"

    # Pula a coleta se o arquivo já foi salvo anteriormente
    if arquivo_existe(nome_arquivo):
        print(f"Arquivo {nome_arquivo} já existe. Pulando coleta...")
        continue

    dfs = []  # Lista para armazenar DataFrames por ano/subopção
    intervalos = anos if config["requer_ano"] else [None]

    # Itera sobre os anos (se aplicável) e subopções da página
    for ano in intervalos:
        for subopcao in config["subopcoes"]:
            params = {"opcao": opcao}
            if ano is not None:
                params["ano"] = ano
            if subopcao:
                params["subopcao"] = subopcao

            print(f"Coletando: opcao={opcao}, ano={ano}, subopcao={subopcao}")
            tabelas, soup = scrape_tabelas(base_url, params)

            for i, tabela in enumerate(tabelas):
                try:
                    # Decide como processar a tabela com base na estrutura do HTML
                    if soup and (soup.find('td', {'class': 'tb_item'}) or tabela.find('td', {'class': 'tb_item'})):
                        df = processar_tabela_com_itens(soup, tabela)
                    else:
                        df = pd.read_html(str(tabela))[0]

                    # Limpeza e tratamento de dados nulos
                    df = tratar_valores_nulos(df)

                    # Adiciona coluna do ano (se aplicável)
                    if ano is not None:
                        df['Ano'] = ano

                    dfs.append(df)
                except Exception as e:
                    print(f"Erro ao processar tabela: {str(e)}")

            sleep(2)  # Pausa entre requisições para respeitar o servidor

    # Salva o arquivo CSV final se houve coleta
    if dfs:
        caminho_completo = os.path.join(DATA_DIR, nome_arquivo)
        df_final = pd.concat(dfs, ignore_index=True)
        df_final.to_csv(caminho_completo, index=False)
        print(f"Dados salvos em {caminho_completo}")

print("Coleta concluída!")


Coletando: opcao=opt_02, ano=2008, subopcao=None


<ipython-input-2-b75a3027446f>:87: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


Coletando: opcao=opt_02, ano=2009, subopcao=None


<ipython-input-2-b75a3027446f>:87: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


Coletando: opcao=opt_02, ano=2010, subopcao=None


...

Coletando: opcao=opt_06, ano=2023, subopcao=subopt_01


<ipython-input-2-b75a3027446f>:125: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tabela))[0]
<ipython-input-2-b75a3027446f>:87: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


Coletando: opcao=opt_06, ano=2023, subopcao=subopt_02


<ipython-input-2-b75a3027446f>:125: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tabela))[0]
<ipython-input-2-b75a3027446f>:87: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


Dados salvos em data/raw/exportacao_2008_2023.csv
Coleta concluída!
